In [ ]:
# install libraries
!pip install bitsandbytes==0.42.0 -q
!pip install datasets==2.17.0 -q
!pip install accelerate==0.27.2 -q
!pip install transformers==4.37.0 -q
!pip install evaluate==0.4.0 -q
!pip install autoawq==0.2.2 -q
!pip install langchain==0.1.8 -q

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, pipeline
import torch
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
import evaluate
import os
import json
import re

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.8/76.8 kB 2.8 MB/s eta 0:0

In [ ]:
# Quantization configuration for mistral7b

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

model_4bit = AutoModelForCausalLM.from_pretrained( "mistralai/Mistral-7B-Instruct-v0.1", device_map="auto",quantization_config=quantization_config, )
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
pipeline_inst = pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=2500,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

In [ ]:
llm = HuggingFacePipeline(pipeline=pipeline_inst)

In [ ]:
template = """[INST] You are an respectful and helpful assistant, respond always be precise, assertive and politely answer in few words conversational english.
Answer the question below from context below :
{question} [/INST]
"""

def generate_response(question):
  prompt = PromptTemplate(template=template, input_variables=["question","context"])
  llm_chain = LLMChain(prompt=prompt, llm=llm)
  response = llm_chain.invoke({"question":question})
  return response

In [ ]:
generate_response("If I have 2 apples and 6 bananas, how much fruit do I have?")

{'question': 'Name one president of america?',
 'text': '\nOne president of America is George Washington.'}

In [ ]:
# SELECT DATASET
#dataset_name = "aqua_rat"
dataset_name = "gsm8k"

def generate_responses(dataset_name, num_questions):
  dataset = load_dataset(dataset_name, 'main', split='test')
  responses = []

  for i in range(num_questions):

    question_data = dataset[i]
    question = question_data['question']
    correct_answer = question_data['answer']

    generated_answer = generate_response(question)

    responses.append({
        'question' : question,
        'response' : generated_answer,
        'correct'  : correct_answer
    })

  return responses

responses = generate_responses(dataset_name="gsm8k", num_questions=5)


with open('responses.json', 'w') as f:
  json.dump(responses, f, indent=4)


Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [ ]:
# Load the JSON file into a Python variable
file_path = 'responses.json'  # Path to your JSON file
with open(file_path, 'r') as file:
    responses = json.load(file)

# Print the loaded JSON data with indentation for readability
print(json.dumps(responses, indent=4))

[
    {
        "question": "Janet\u2019s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
        "response": {
            "question": "Janet\u2019s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
            "text": "\nJanet makes $4 per day at the farmers' market by selling the remainder eggs after breakfast and muffins."
        },
        "correct": "Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer\u2019s market.\n#### 18"
    },
    {
        "question": "A robe takes 2 b

In [ ]:
exact_match = evaluate.load("exact_match")

In [ ]:
# Function to load responses from the JSON file
def load_responses(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)


# Function to preprocess responses
def preprocess_responses(responses):
    preprocessed_responses = []
    for response in responses:
        question = response['question']
        model_response = response['response']['text']  # Extract text from the response dictionary
        correct_answer = response['correct']

        # Function to process a text field to extract the last numeric value
        def process_text_field(text):
            # Remove commas from numeric answers
            processed_text = re.sub(r"(\d),(\d)", r"\1\2", text)
            # Find all numeric values (potentially with decimals) in the text
            numbers = re.findall(r"[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?", processed_text)
            # Convert the last found number to float if any numbers are found
            if numbers:
                return float(numbers[-1].replace(',', ''))
            else:
                return processed_text  # Return original text if no numbers found

        # Process both the model response and the correct answer
        numeric_response = process_text_field(model_response)
        numeric_correct_answer = process_text_field(correct_answer)

        # Append the preprocessed response with processed correct answer
        preprocessed_responses.append({
            'question': question,
            'response': numeric_response,  # Possibly include numeric response
            'correct': numeric_correct_answer  # Processed correct answer
        })

    return preprocessed_responses

# Example usage
file_path = 'responses.json'  # Path to your responses.json file
responses = load_responses(file_path)
preprocessed_responses = preprocess_responses(responses)

# Optionally, save the preprocessed responses back to a file
with open('preprocessed_responses.json', 'w') as file:
    json.dump(preprocessed_responses, file, indent=4)


In [ ]:
print(json.dumps(preprocessed_responses, indent=4))

[
    {
        "question": "Janet\u2019s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
        "response": 4.0,
        "correct": 18.0
    },
    {
        "question": "A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?",
        "response": 3.0,
        "correct": 3.0
    },
    {
        "question": "Josh decides to try flipping a house.  He buys a house for $80,000 and then puts in $50,000 in repairs.  This increased the value of the house by 150%.  How much profit did he make?",
        "response": 75000.0,
        "correct": 70000.0
    },
    {
        "question": "James decides to run 3 sprints 3 times a week.  He runs 60 meters each sprint.  How many total meters does he run a week?",
        "res

In [ ]:
# n-shot version

In [ ]:
# calculate accuracy // em score
exact_match = evaluate.load("exact_match")
predictions = [str(item["response"]) for item in preprocessed_responses]
targets = [str(item["correct"]) for item in preprocessed_responses]

em_score = exact_match.compute(predictions=predictions, references=targets, ignore_case=True, ignore_punctuation=True)["exact_match"]

In [ ]:
print(em_score)

0.2




---

Below is wizard math approach, runs out of memory on T4 though if using CUDA, and very slow if using CPU for inference

In [ ]:
# SET MODEL
model_name = "WizardLM/WizardMath-7B-V1.1"
#model_name = "TheBloke/WizardMath-13B-V1.0-GGUF"

# Tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Check if GPU is available and move the model to GPU
if torch.cuda.is_available():
    model = model.to('cuda')
    print('Using GPU...')
else:
    print("GPU is not available, using CPU instead.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/948 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

GPU is not available, using CPU instead.


In [ ]:
def generate_answer(question, maxLength = 500):

  # tokenise model inputs
  inputs = tokenizer.encode(question, return_tensors="pt")

  # generate model outputs
  outputs = model.generate(inputs, max_length=maxLength, num_return_sequences=1)

  # decode
  answer = tokenizer.decode(output[0], skip_special_tokens=True)

  return answer

In [ ]:
# Test
test_answer = generate_answer("Hello, how are you")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

In [ ]:
print(test_answer)

In [ ]:
# SELECT DATASET
dataset_name = "aqua_rat"

# NO. OF QUESTIONS
num_questions = 5

dataset = load_dataset(dataset_name, split='test')
responses = []

# Go through questions
for i in range(num_questions):

  # Data
  question_data = dataset[i]
  question = question_data['question']
  correct_answer = question_data['correct']

  # Generate response
  answer = generate_answer(question)

  # Output in json
  responses.append({
      'question' : question,
      'response' : answer,
      'correct'  : correct_answer
  })


with open('responses.json', 'w') as f:
  json.dump(responses, f, indent=4)


In [ ]:
# Evaluation
import evaluate
import os
import json
import re

# Load responses from the JSON file
def load_responses(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

# Preprocess responses
def preprocess_responses(responses):
    preprocessed_responses = []
    for response in responses:
        question = response['question']
        model_response = response['response']
        correct_answer = response['correct']

        # Remove commas from numeric answers in the model's response
        preprocessed_model_response = re.sub(r"(\d),(\d)", r"\1\2", model_response)

        # Attempt to convert the processed model response to a float if it's numeric
        try:
            assert float(preprocessed_model_response), f"Response is not a valid number: {preprocessed_model_response}"
            numeric_response = float(preprocessed_model_response)
        except ValueError:
            numeric_response = preprocessed_model_response  # Keep as string if not numeric

        # Append the preprocessed response
        preprocessed_responses.append({
            'question': question,
            'response': numeric_response,
            'correct': correct_answer
        })

    return preprocessed_responses

# Example usage
file_path = 'responses.json'  # Path to your responses.json file
responses = load_responses(file_path)
preprocessed_responses = preprocess_responses(responses)

# Optionally, save the preprocessed responses back to a file
with open('preprocessed_responses.json', 'w') as file:
    json.dump(preprocessed_responses, file, indent=4)
